# Aula9_reducao_dimensionalidade_ACP_ACM

In [ ]:
library(tidyverse)
library(FactoMineR)
library(stats)
library(scatterplot3d)
library(IRdisplay)

#### O conceito: o fundamento em algebra linear e as três categorias de análises que abordaremos nas próximas aulas

In [ ]:
display_png(file="Aula9_example_dimensional_reduction.png")
print("fonte: https://medium.com/free-code-camp/an-overview-of-principal-component-analysis-6340e3bc4073")
display_png(file="Aula9_machine_learning_influence.png")
print("fonte: https://www.datasciencecentral.com/profiles/blogs/machine-learning-summarized-in-one-picture")

#### Para mais noções acerca do conceito, abra no seu computador agora essa página de ilustração para conversarmos a respeito:

https://dododas.github.io/linear-algebra-with-python/posts/16-12-29-2d-transformations.html

### Análises de Correspondência Múltipla e Análises de Componentes Principais: aspectos fundamentais.  


Ambas análises são voltadas para percepção de interdependências entre as variáveis. Ambas possuem foco sobre a covariância do conjunto dos dados (em relação a um centróide adquirido após transformações lineares).


Sequencia de passos:
- 1º Busca-se o centro dos dados: xi,a =  xi,a – μ;
- 2º computa-se a covariância da matriz (soma das distâncias dos quadrados, para se chegar ao eigenvalue/autovalor);
- 3º recolhe-se as dimensões com maiores cargas de autovalores (que possuem as maiores (co)variâncias explicadas do conjunto dos valores) para análise detalhada;
- 4º para análises mais detalhadas, observa-se as contribuições das variáveis para cada dimensão e "vasculha-se" as especificidades de variável e dimensão;
- 5º analisa-se o conjunto das dispersões das variáveis em relação ao centróide e observa-se suas interdependências.


Em suma: ACP/ACM reduz dimensionalidade para se analisar a covariância (pergunta respondida por ACP/ACM: quais as variáveis com maior variação em relação ao conjunto dos dados/centróide?)

### Exemplo com ACM

Amostra de 57 casos de nomeações para secretarias do Ministério da Agricultura, Pecuária e Abastecimento (MAPA) entre 1995 e 2018.

Problema de investigação exploratória: dimensões difusas de impactos Estado-sociedade em relação às nomeações no MAPA (pergunta: quais aspectos societais relacionam-se com a ocupação de cargos no MAPA?)

6 variáveis interdependentes sob análise:

- Se o(a) nomeado(a) possuiu circulação por alguma organização patronal rural previamente à nomeação (duas categorias: 'patronal' ou 'nao' -- recod: 'Patr', 'ñPatr');
- qual o partido no governo no momento da nomeação (três categorias: 'PSDB', 'PT' ou 'PMDB');
- se o(a) nomeado(a) possuía filiação partidária no momento da nomeação (duas categorias'sim' e 'nao' -- recod: 'Part', 'ñPart');
- nível de formação do(a) nomeado(a) (três categorias: 'ensino_medio', 'graduacao_ou_especializacao' e 'mestrado_ou_doutorado'  -- recod: 'edEM', 'edGouE' e 'edMouD');
- tipo de secretaria que acolheu a nomeação (duas categorias: 'execucao_de_pol_pub' 'planejamento_estrategico' -- recod: 'tsExPPub', 'tsPlEs');
- perfil de carreira profissional prévia à nomeação (quatro categorias: 'Burocrata_outsider', 'Mercado', 'Burocrata_insider' e 'Academia' -- recod: 'BuIn', 'Merc', 'BuOu', 'Acad').

In [ ]:
df <- read_csv("Aula9_reducao_dimensionalidade_ACM.csv")



unique(df$circulacao_organizacional_externa_patronal_sindical_ou_organismo)
unique(df$partido_nom)
unique(df$filiacao_partidaria)
unique(df$nivel_escolaridade)
unique(df$tipo_de_secretaria)
unique(df$emprego_cargo_instituicao_trajet_pre_resumido2)
nrow(df)
df

#### Estruturação da Análise de Correspondência Múltipla (processamento de redução de dimensionalidade):

In [ ]:
acm <- data.frame(df$circulacao_organizacional_externa_patronal_sindical_ou_organismo,
df$filiacao_partidaria,
df$partido_nom , df$nivel_escolaridade,
df$tipo_de_secretaria,
df$emprego_cargo_instituicao_trajet_pre_resumido2)

result <- MCA(acm, ncp = 10, ind.sup = 1, quanti.sup = NULL, 
               quali.sup = NULL, excl=NULL, graph = TRUE, 
               level.ventil = 0, axes = c(1,2), row.w = NULL, 
               method="Indicator", na.method="NA", tab.disj=NULL)
result


##### Banco proveniente da redução de dimensionalidade e análise das contribuições das categorias (agora variáveis) para cada uma das dimensões -- quando MAIS ALTO O VALOR DE CONTRIBUIÇÃO, MAIS ASSOCIADA A VARIÁVEL/CATEGORIA À DIMENSÃO (importante para análise de quem busca detalhar aspectos relativos de variável):

In [ ]:
dfacm <- as.data.frame(result$var$coord)
dfacm

contribuicoes_var <- as.data.frame(result$var$contrib)
contribuicoes_var

#### 2. Avaliação de Elbow Plot: quando menos dimensões contemplar, melhor o ajuste sobre os valores individuais (porém menor o percentual de variância explicada)

In [ ]:
an_var_autovalores <- as.data.frame(result$eig)
colnames(an_var_autovalores)[1] <- "autovalores"
colnames(an_var_autovalores)[2] <- "porcent_variancia"
colnames(an_var_autovalores)[3] <- "porcent_acum_variancia"
an_var_autovalores

barplot(height=an_var_autovalores$porcent_variancia, names=rownames(an_var_autovalores), 
        xlab="dimensões", 
        ylab="% de variância explicada", 
        main="Elbow Plot: (co)variancias explicadas", 
        ylim=c(0,25)
        )

#### 3. Análise das dispersões das variáveis  e suas interdependências (oito dimensões com impacto sobre o conjunto dos dados):

In [ ]:
dfacmplot <- as.data.frame(dfacm)
labels <- c('ñPatr', 'Patr', 'ñPart', 'Part', 'govPMDB', 'govPSDB', 'govPT', 'edEM', 'edGouE', 'edMouD', 'tsExPPub', 'tsPlEs', 
           'Acad', 'BuIn', 'BuOu', 'Merc')
dfacmplot

colnames(dfacmplot)[1] <- "Dim_1"
colnames(dfacmplot)[2] <- "Dim_2"
colnames(dfacmplot)[3] <- "Dim_3"
colnames(dfacmplot)[4] <- "Dim_4"
colnames(dfacmplot)[5] <- "Dim_5"
colnames(dfacmplot)[6] <- "Dim_6"
colnames(dfacmplot)[7] <- "Dim_7"
colnames(dfacmplot)[8] <- "Dim_8"

plot(dfacmplot$Dim_1, dfacmplot$Dim_2, main= "Impactos Estado-sociedade em relação às nomeações no MAPA", col="lightblue", pch=19, cex=2)
text(dfacmplot$Dim_1, dfacmplot$Dim_2, labels=labels, cex= 1)

#### Gráfico em 3d para se ter noção da redução de dimensionalidade:

In [ ]:
scatterplot3d(dfacmplot$Dim_1, dfacmplot$Dim_3, dfacmplot$Dim_2, main= "Análise de nomeações para o MAPA (1995-2018)", , pch = 16, color="steelblue")
text(dfacmplot$Dim_1, dfacmplot$Dim_3, dfacmplot$Dim_2, labels = labels,
     cex= 0.5, col = "steelblue")

É possível perceber que a variável ACADEMIA é demasiadamente discrepante. É possível fazer a sua retirada para melhorar o gráfico ("perder informação para ganhar precisão"). Para isso, é necessário checar também qual é a dimensão na qual ele mais contribui para a variância e retirá-la também -- pós-checagem, verificou-se ser a dimensão 5). Let's go...

In [ ]:
dfacmplot <- as.data.frame(dfacm[-c(13),])
labels <- c('ñPatr', 'Patr', 'ñPart', 'Part', 'govPMDB', 'govPSDB', 'govPT', 'edEM', 'edGouE', 'edMouD', 'tsExPPub', 'tsPlEs', 
            'BuIn', 'BuOu', 'Merc')
dfacmplot

colnames(dfacmplot)[1] <- "Dim_1"
colnames(dfacmplot)[2] <- "Dim_2"
colnames(dfacmplot)[3] <- "Dim_3"
colnames(dfacmplot)[4] <- "Dim_4"
colnames(dfacmplot)[6] <- "Dim_6"
colnames(dfacmplot)[7] <- "Dim_7"
colnames(dfacmplot)[8] <- "Dim_8"

plot(dfacmplot$Dim_1, dfacmplot$Dim_2, main= "Impactos Estado-sociedade em relação às nomeações no MAPA", col="lightblue", pch=19, cex=2)
text(dfacmplot$Dim_1, dfacmplot$Dim_2, labels=labels, cex= 1)